In [1]:
import torch, csv, emoji
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sigmoid
import torch.optim as optim
from torchtext.data.metrics import bleu_score
import numpy as np
from transformers import AutoModel, AutoTokenizer 
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer

from Bert_NN import read_csv, vectorize_sentences, convert_str, emojiDataset, toEmoji
from utils import DATAPATH

Using cpu device


In [2]:
torch.cuda.is_available()

False

In [3]:
log = open('log.txt', 'w', encoding='utf-8')

In [4]:
features, labels, total, sentences, emojis, labels_classes = vectorize_sentences(DATAPATH)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



-----------------------Begining Data Loading-----------------------

Loading data for file '01'.
Loaded 240 labelled sentence and emoji data.

Loading data for file '02'.
Loaded 227 labelled sentence and emoji data.

Loading data for file '03'.
Loaded 247 labelled sentence and emoji data.

Loading data for file '04'.
Loaded 248 labelled sentence and emoji data.

Loading data for file '05'.
Loaded 205 labelled sentence and emoji data.

Loaded total 1167 labelled sentence and emoji data in 0.0 minutes.

-----------------------Finished Data Loading-----------------------


-----------------------Begining Feature Extraction-----------------------
Extracted total 1167 feature and label data in 0.9 minutes.

-----------------------Finished Feature Extraction-----------------------



In [9]:
loss_func = nn.BCELoss()
S = Sigmoid()
epochs = 10
num_folds = 5
cross_validation = model_selection.KFold(n_splits=num_folds, random_state=None, shuffle=True)

bleu_max_n = 1
bleu_weights = [1]

best_to_emoji = None
best_score = -1

In [10]:
for index, (train_index, test_index) in enumerate(cross_validation.split(features)):
    print(f'\nFold: {index+1}/{num_folds}')
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    train_dataset = emojiDataset(features_train, labels_train)
    data_loader = DataLoader(dataset=train_dataset, batch_size=32)

    toemoji = toEmoji()
    learn_rate = optim.Adam(toemoji.parameters(), lr=0.005)

    for i in range(epochs):
        # print(f'\tEpoch: {i+1}/{epochs}')
        for data in data_loader:
            cur_train_features, cur_train_labels = data
            toemoji.zero_grad()
            results = toemoji(cur_train_features.float())
            S_results = S(results)
            loss = loss_func(S_results, cur_train_labels)
            loss.backward()
            learn_rate.step()
        # print('\t\tloss is: {:.4f}'.format(loss))
    
    test_dataset = emojiDataset(features_test, labels_test)
    data_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)
    total_BLEU = 0
    for data in data_loader:
        cur_test_features, cur_test_labels = data
        toemoji.zero_grad()
        results = toemoji(cur_test_features.float())
        result_labels = []
        for result_index, result in enumerate(results) :
            result_labels.append(['1' if elem.item()>0 else '0' for elem in result])        
        total_BLEU += bleu_score(convert_str(cur_test_labels), result_labels, max_n=bleu_max_n, weights=bleu_weights)
    average_BLUE = total_BLEU/len(data_loader)
    print(f'\nFold: {index+1}/{num_folds} with Average BLEU Score : {average_BLUE}')
    if average_BLUE > best_score: best_to_emoji = toemoji


Fold: 0/5

Fold: 0/5 with Average BLEU Score : 0.0

Fold: 1/5

Fold: 1/5 with Average BLEU Score : 0.0

Fold: 2/5

Fold: 2/5 with Average BLEU Score : 0.0

Fold: 3/5

Fold: 3/5 with Average BLEU Score : 0.0

Fold: 4/5

Fold: 4/5 with Average BLEU Score : 0.0


In [8]:
from collections import defaultdict

collection = defaultdict(int)
predicted = defaultdict(int)

total_dataset = emojiDataset(features, labels)
data_loader = DataLoader(dataset=total_dataset, batch_size=32, shuffle=True)

with torch.no_grad():
    for i, data in enumerate(total_dataset) :
        feature, label = data
        result = toemoji(feature.float())
        cur_BLEU = bleu_score(result, label)
        result = [index for index, elem in enumerate(result) if elem.item()>0]
        original_label_index = [index for index, elem in enumerate(label) if elem!=0]
        # print(original_label_index)
        for label_index in result:
            predicted[label_index] += 1
        for label_index in original_label_index:
            collection[label_index] += 1
        
        real_labels = [labels_classes[label_index] for label_index in result]

        print(f'Sentence: {sentences[i]} with Bleu score: {cur_BLEU}\nOriginal labels: {emojis[i]}, output labels: {real_labels}\n') 

for index, label in enumerate(labels_classes):
    print(f'Emoji: {label} has count: {collection[index]} predicted times: {predicted[index]}')

        

TypeError: len() of a 0-d tensor